In [56]:
#import all important modules

from sklearn.externals import joblib

from skimage.feature import hog
from sklearn.svm import LinearSVC
import numpy as np
import pandas as pd
from sklearn.datasets import fetch_mldata
from scipy.io import loadmat

In [57]:
mnist  = loadmat('mnist-original.mat')

In [58]:
mnist_data = mnist["data"].T
mnist_label = mnist["label"][0]

In [59]:
mnist_data.shape

(70000, 784)

In [60]:
mnist_label.shape

(70000,)

In [61]:
features = np.array(mnist_data,'int16')
labels = np.array(mnist_label,'int')

In [62]:
list_hog_feature = []
for feature in features:
    fd = hog(feature.reshape((28,28)),orientations=9,pixels_per_cell=(14,14),cells_per_block=(1,1),visualise=False)
    list_hog_feature.append(fd)
hog_features = np.array(list_hog_feature,'float64')    

C:\Users\IamChamp\Anaconda3\lib\site-packages\skimage\feature\_hog.py:119: skimage_deprecation: Default value of `block_norm`==`L1` is deprecated and will be changed to `L2-Hys` in v0.15
  'be changed to `L2-Hys` in v0.15', skimage_deprecation)


In [63]:
#Define the Classifier

clf  = LinearSVC()

In [64]:
#Fit the data

clf.fit(hog_features,labels)

LinearSVC(C=1.0, class_weight=None, dual=True, fit_intercept=True,
     intercept_scaling=1, loss='squared_hinge', max_iter=1000,
     multi_class='ovr', penalty='l2', random_state=None, tol=0.0001,
     verbose=0)

In [66]:
joblib.dump(clf,'digit_classifier1.pkl',compress=3)

['digit_classifier1.pkl']

In [103]:
##Testing the Classifier
import cv2
from sklearn.externals import joblib
from skimage.feature import hog
import numpy as np

In [104]:
##Load The Classifier

clf = joblib.load('digit_classifier1.pkl')

In [152]:
##Reading the input image

image = cv2.imread("digit_Reco_sample.jpg")
#cv2.imshow("Showing sample image",image)
#cv2.waitKey()

In [153]:
# Convert to grayscale and apply Gaussian filtering
image_gray = cv2.cvtColor(image,cv2.COLOR_BGR2GRAY)
image_gray = cv2.GaussianBlur(image_gray,(5,5),0)
#cv2.imshow("Showing sample image",image_gray)
#cv2.waitKey()

In [154]:
#Threshold the image
ret,image_thresh = cv2.threshold(image_gray,90,255,cv2.THRESH_BINARY_INV)
#print(ret)

In [155]:
#Finding Contours into the image
image_cnt, contours, _ = cv2.findContours(image_thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)


In [156]:

# Get rectangles contains each contour
rects = [cv2.boundingRect(image_cnt) for ctr in image_cnt]


In [157]:
# For each rectangular region, calculate HOG features and predict
# the digit using Linear SVM.

for rect in rects:
    cv2.rectangle(image, (rect[0], rect[1]), (rect[0] + rect[2], rect[1] + rect[3]), (0, 255, 0), 2)   
       # Make the rectangular region around the digit
    leng = int(rect[3] * 1.6)
    pt1 = int(rect[1] + rect[3] // 2 - leng // 2)
    pt2 = int(rect[0] + rect[2] // 2 - leng // 2)
    roi = image_thresh[pt1:pt1+leng, pt2:pt2+leng]
        # Resize the image
    roi = cv2.resize(roi, (28, 28), interpolation=cv2.INTER_AREA)
    roi = cv2.dilate(roi, (3, 3))
        # Calculate the HOG features
    roi_hog_fd = hog(roi, orientations=9, pixels_per_cell=(14, 14), cells_per_block=(1, 1), visualise=False)
    nbr = clf.predict(np.array([roi_hog_fd], 'float64'))
    cv2.putText(image, str(int(nbr[0])), (rect[0], rect[1]),cv2.FONT_HERSHEY_DUPLEX, 2, (0, 255, 255), 3)
cv2.imshow("Resulting Image with Rectangular ROIs", image)
cv2.waitKey()    

C:\Users\IamChamp\Anaconda3\lib\site-packages\skimage\feature\_hog.py:119: skimage_deprecation: Default value of `block_norm`==`L1` is deprecated and will be changed to `L2-Hys` in v0.15
  'be changed to `L2-Hys` in v0.15', skimage_deprecation)


-1